- 시작하자마자  drive mounting 하기!!

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# # colab에서 셀레니움 실행하기(처음 한번만)
# !pip install selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [11]:
# 모듈 import
import time
import sys 
import os 
import re

import pandas as pd
import numpy as np

from selenium import webdriver 
# import chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


import urllib.request as req
from bs4 import BeautifulSoup


import warnings
warnings.filterwarnings('ignore')

# 경로 지정
- 시작시 변경 필요!!!!

In [3]:
driver_path = "/content/drive/My Drive/projects/KlueBERT/chromedriver.exe"
csv_path = "/content/drive/My Drive/projects/KlueBERT/"

os.chdir(csv_path)

# 검색 지역 설정

## 지역명 DB 불러오기

In [4]:
loc_df = pd.read_csv('korea_location.csv')
loc_df[:10]

,location_no,location,place1_no,place1,place2_no,place2,place
0,11,서울특별시,11010,종로구,11010720,청운효자동,종로구 청운효자동
1,11,서울특별시,11010,종로구,11010530,사직동,종로구 사직동
2,11,서울특별시,11010,종로구,11010540,삼청동,종로구 삼청동
3,11,서울특별시,11010,종로구,11010550,부암동,종로구 부암동
4,11,서울특별시,11010,종로구,11010560,평창동,종로구 평창동
5,11,서울특별시,11010,종로구,11010570,무악동,종로구 무악동
6,11,서울특별시,11010,종로구,11010580,교남동,종로구 교남동
7,11,서울특별시,11010,종로구,11010600,가회동,종로구 가회동
8,11,서울특별시,11010,종로구,11010610,종로1.2.3.4가동,종로구 종로1.2.3.4가동
9,11,서울특별시,11010,종로구,11010630,종로5.6가동,종로구 종로5.6가동


## 지역 내 카페목록 가져오기

In [12]:
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)


# 리스트변수명 설정
place_list=[]
title_list = []
f_data_list = []

# 수집
for p in range(20):   # ========================================================================================== 테스트용
#for p in range(len(loc_df)): 
    # 검색어 입력    
    query = '{0} 카페'.format(loc_df['place'][p]) #------------지역[칼럼명]으로 데이터마다 수정
    driver.get(f"https://map.naver.com/v5/search/{query}?c=14203933.7141038,4562681.4505997,10,0,0,0,dh")
    time.sleep(2)
    # 검색결과 iframe 접근
    driver.switch_to.frame("searchIframe")
    
    # 처음 돌릴 땐 자료 60개 수집
    # 1페이지로 돌아간 뒤 한번 더 돌리면 300개 정도 추출됨 (검색 결과 많은 경우)

    try: 
        for i in range(1,7): 
            driver.find_element_by_link_text(str(i)).click()
            time.sleep(2)
            
            try: 
                for j in range(3,70,3):
                    element1 = driver.find_elements_by_css_selector('.OXiLu')[j]
                    ActionChains(driver).move_to_element(element1).key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).perform()
                    element2 = driver.find_elements_by_css_selector('._3Apve')[j]
                    ActionChains(driver).move_to_element(element2).key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).perform()
            except:
                pass
            
                
            # 카페명
            if len(driver.find_elements_by_css_selector('.OXiLu')) == 0:
                title_raw = driver.find_elements_by_css_selector('._3Apve')
            else:
                title_raw = driver.find_elements_by_css_selector('.OXiLu')
            for title in title_raw:
                title = title.text
                place_list.append(loc_df['place'][p]) #------------지역[칼럼명]으로 데이터마다 수정
                title_list.append(title)
            
            
#             print(i,'p',end='\t') #몇번째 페이지 크롤링 중인지 확인
    except:
        pass
    print(query,"/ 누적 카페 수 :", len(title_list) )
print("-"*20, "목록 가져오기 종료", "-"*20)

종로구 청운효자동 카페 / 누적 카페 수 : 0
종로구 사직동 카페 / 누적 카페 수 : 0
종로구 삼청동 카페 / 누적 카페 수 : 0
종로구 부암동 카페 / 누적 카페 수 : 0
종로구 평창동 카페 / 누적 카페 수 : 0


KeyboardInterrupt: ignored

In [ ]:
## 데이터 프레임 만들기
list_df = pd.DataFrame({'place':place_list, 'title':title_list})
list_df

,place,title
0,종로구 청운효자동,더숲 초소책방
1,종로구 청운효자동,에디션덴마크쇼룸
2,종로구 청운효자동,아키비스트
3,종로구 청운효자동,서촌스코프
4,종로구 청운효자동,효자베이커리
...,...,...
1175,중구 명동,홀리핸즈커피
1176,중구 명동,오코아 무교점
1177,중구 명동,아운커피 다동점
1178,중구 명동,카페화이트랩


In [ ]:
## csv 저장 
list_df.to_csv(csv_path+"cafe_list.csv", encoding='cp949') 

# 저장 후 카페 리스트에 접근

In [ ]:
## 카페 리스트 load
cafe_df = pd.read_csv(csv_path+"cafe_list.csv", encoding='cp949')
cafe_df

,Unnamed: 0,place,title
0,0,종로구 청운효자동,더숲 초소책방
1,1,종로구 청운효자동,에디션덴마크쇼룸
2,2,종로구 청운효자동,아키비스트
3,3,종로구 청운효자동,서촌스코프
4,4,종로구 청운효자동,효자베이커리
...,...,...,...
1175,1175,중구 명동,홀리핸즈커피
1176,1176,중구 명동,오코아 무교점
1177,1177,중구 명동,아운커피 다동점
1178,1178,중구 명동,카페화이트랩


In [ ]:
# url 담을 컬럼 만들어놓기
cafe_df['search'] = cafe_df['place']+' '+cafe_df['title']
cafe_df['url'] = '' 
cafe_df.drop('Unnamed: 0', axis = 1, inplace = True)
cafe_df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
cafe_df

,place,title,search,url
0,종로구 청운효자동,더숲 초소책방,종로구 청운효자동 더숲 초소책방,
1,종로구 청운효자동,에디션덴마크쇼룸,종로구 청운효자동 에디션덴마크쇼룸,
2,종로구 청운효자동,아키비스트,종로구 청운효자동 아키비스트,
3,종로구 청운효자동,서촌스코프,종로구 청운효자동 서촌스코프,
4,종로구 청운효자동,효자베이커리,종로구 청운효자동 효자베이커리,
...,...,...,...,...
1175,중구 명동,홀리핸즈커피,중구 명동 홀리핸즈커피,
1176,중구 명동,오코아 무교점,중구 명동 오코아 무교점,
1177,중구 명동,아운커피 다동점,중구 명동 아운커피 다동점,
1178,중구 명동,카페화이트랩,중구 명동 카페화이트랩,


# url 찾기 for문

In [ ]:
#### 여러개가 나오면, url을 가져올 수 없음

In [ ]:
for i, keyword in enumerate(cafe_df['search'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {cafe_df.shape[0]} 행", keyword)
    try:
        naver_map_search_url = f'https://map.naver.com/v5/search/+{keyword}/place' # 검색 url 만들기
        driver.get(naver_map_search_url) # 검색 url 접속, 즉 검색하기 
        time.sleep(4)  # 중요함, 중간에 500에러 발생 방지 위함 
        cu = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지 
        res_code = re.findall(r"place/(\d+)", cu)
        final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#'
        time.sleep(0.5)
        print(final_url)
        cafe_df['url'][i]=final_url
    except IndexError: 
        time.sleep(0.5)
        cafe_df['url'][i]= '' 
        print('none') 
print("-"*30, '종료', '-'*30)

이번에 찾을 키워드 : 0 / 1179 행 종로구 청운효자동 더숲 초소책방
https://pcmap.place.naver.com/restaurant/1602048207/review/visitor#
이번에 찾을 키워드 : 1 / 1179 행 종로구 청운효자동 에디션덴마크쇼룸
https://pcmap.place.naver.com/restaurant/1787227960/review/visitor#
이번에 찾을 키워드 : 2 / 1179 행 종로구 청운효자동 아키비스트
https://pcmap.place.naver.com/restaurant/1783153281/review/visitor#
이번에 찾을 키워드 : 3 / 1179 행 종로구 청운효자동 서촌스코프
https://pcmap.place.naver.com/restaurant/462568628/review/visitor#
이번에 찾을 키워드 : 4 / 1179 행 종로구 청운효자동 효자베이커리
https://pcmap.place.naver.com/restaurant/17898357/review/visitor#
이번에 찾을 키워드 : 5 / 1179 행 종로구 청운효자동 통인스윗
none
이번에 찾을 키워드 : 6 / 1179 행 종로구 청운효자동 카페이마
https://pcmap.place.naver.com/restaurant/11678597/review/visitor#
이번에 찾을 키워드 : 7 / 1179 행 종로구 청운효자동 베어카페
none
이번에 찾을 키워드 : 8 / 1179 행 종로구 청운효자동 인왕산 대충유원지
https://pcmap.place.naver.com/restaurant/1077382386/review/visitor#
이번에 찾을 키워드 : 9 / 1179 행 종로구 청운효자동 스쿠퍼젤라또
https://pcmap.place.naver.com/restaurant/1553059011/review/visitor#
이번에 찾을 키워드 : 10 / 1179 행 종로구 청운효자동 할리스 세종로점

https://pcmap.place.naver.com/restaurant/32587763/review/visitor#
이번에 찾을 키워드 : 87 / 1179 행 종로구 사직동 내자땅콩
https://pcmap.place.naver.com/restaurant/1234800567/review/visitor#
이번에 찾을 키워드 : 88 / 1179 행 종로구 사직동 LP소리골
https://pcmap.place.naver.com/restaurant/1236906682/review/visitor#
이번에 찾을 키워드 : 89 / 1179 행 종로구 사직동 커피붕붕 커피볶는집
https://pcmap.place.naver.com/restaurant/31481759/review/visitor#
이번에 찾을 키워드 : 90 / 1179 행 종로구 사직동 플레인
none
이번에 찾을 키워드 : 91 / 1179 행 종로구 사직동 카페광화문
none
이번에 찾을 키워드 : 92 / 1179 행 종로구 사직동 히든커피
https://pcmap.place.naver.com/restaurant/1772406863/review/visitor#
이번에 찾을 키워드 : 93 / 1179 행 종로구 사직동 카페자스 광화문점
none
이번에 찾을 키워드 : 94 / 1179 행 종로구 사직동 스타벅스 정동점
none
이번에 찾을 키워드 : 95 / 1179 행 종로구 사직동 와프
https://pcmap.place.naver.com/restaurant/1754811858/review/visitor#
이번에 찾을 키워드 : 96 / 1179 행 종로구 사직동 테이스트메이드 TASTEMADE
https://pcmap.place.naver.com/restaurant/1082403377/review/visitor#
이번에 찾을 키워드 : 97 / 1179 행 종로구 사직동 슬로우레시피
https://pcmap.place.naver.com/restaurant/1410501334/review/vi

https://pcmap.place.naver.com/restaurant/1125403542/review/visitor#
이번에 찾을 키워드 : 176 / 1179 행 종로구 삼청동 어라운드 시소
https://pcmap.place.naver.com/restaurant/1040397882/review/visitor#
이번에 찾을 키워드 : 177 / 1179 행 종로구 삼청동 베르트
https://pcmap.place.naver.com/restaurant/1200438769/review/visitor#
이번에 찾을 키워드 : 178 / 1179 행 종로구 삼청동 aoc
https://pcmap.place.naver.com/restaurant/1196084755/review/visitor#
이번에 찾을 키워드 : 179 / 1179 행 종로구 부암동 부빙
none
이번에 찾을 키워드 : 180 / 1179 행 종로구 부암동 몽핀
https://pcmap.place.naver.com/restaurant/1358846917/review/visitor#
이번에 찾을 키워드 : 181 / 1179 행 종로구 부암동 산모퉁이
https://pcmap.place.naver.com/restaurant/12781756/review/visitor#
이번에 찾을 키워드 : 182 / 1179 행 종로구 부암동 부암동스코프
https://pcmap.place.naver.com/restaurant/37844910/review/visitor#
이번에 찾을 키워드 : 183 / 1179 행 종로구 부암동 부암동 카페 스탐티쉬
https://pcmap.place.naver.com/restaurant/19882640/review/visitor#
이번에 찾을 키워드 : 184 / 1179 행 종로구 부암동 클럽에스프레소
https://pcmap.place.naver.com/restaurant/11694272/review/visitor#
이번에 찾을 키워드 : 185 / 1179 행 종로구 부

https://pcmap.place.naver.com/restaurant/1582751650/review/visitor#
이번에 찾을 키워드 : 260 / 1179 행 종로구 평창동 갤러리 커피하우스
https://pcmap.place.naver.com/restaurant/49949556/review/visitor#
이번에 찾을 키워드 : 261 / 1179 행 종로구 평창동 레이지
none
이번에 찾을 키워드 : 262 / 1179 행 종로구 평창동 R.ed 아리디
https://pcmap.place.naver.com/restaurant/1228754222/review/visitor#
이번에 찾을 키워드 : 263 / 1179 행 종로구 평창동 밀롱가
https://pcmap.place.naver.com/restaurant/37879725/review/visitor#
이번에 찾을 키워드 : 264 / 1179 행 종로구 평창동 헤이마
https://pcmap.place.naver.com/restaurant/37902818/review/visitor#
이번에 찾을 키워드 : 265 / 1179 행 종로구 평창동 카페350
none
이번에 찾을 키워드 : 266 / 1179 행 종로구 평창동 업스테어카페
https://pcmap.place.naver.com/restaurant/610071494/review/visitor#
이번에 찾을 키워드 : 267 / 1179 행 종로구 평창동 아마스빈 상명대점
none
이번에 찾을 키워드 : 268 / 1179 행 종로구 평창동 이디야커피 상명대점
none
이번에 찾을 키워드 : 269 / 1179 행 종로구 평창동 커피식스주스식스
none
이번에 찾을 키워드 : 270 / 1179 행 종로구 평창동 dropp
https://pcmap.place.naver.com/restaurant/36113914/review/visitor#
이번에 찾을 키워드 : 271 / 1179 행 종로구 평창동 카페SMC 상명대점
https://p

https://pcmap.place.naver.com/restaurant/1553059011/review/visitor#
이번에 찾을 키워드 : 352 / 1179 행 종로구 교남동 서촌금상고로케
https://pcmap.place.naver.com/restaurant/37996721/review/visitor#
이번에 찾을 키워드 : 353 / 1179 행 종로구 교남동 모브
https://pcmap.place.naver.com/restaurant/1187495648/review/visitor#
이번에 찾을 키워드 : 354 / 1179 행 종로구 교남동 베이커스퍼센트
https://pcmap.place.naver.com/restaurant/1242831532/review/visitor#
이번에 찾을 키워드 : 355 / 1179 행 종로구 교남동 통의동단팥
https://pcmap.place.naver.com/restaurant/35272163/review/visitor#
이번에 찾을 키워드 : 356 / 1179 행 종로구 교남동 라그린
https://pcmap.place.naver.com/restaurant/1310154588/review/visitor#
이번에 찾을 키워드 : 357 / 1179 행 종로구 교남동 침니펍
https://pcmap.place.naver.com/restaurant/1337009934/review/visitor#
이번에 찾을 키워드 : 358 / 1179 행 종로구 교남동 세이지핀치
https://pcmap.place.naver.com/restaurant/1276434255/review/visitor#
이번에 찾을 키워드 : 359 / 1179 행 종로구 교남동 모카
none
이번에 찾을 키워드 : 360 / 1179 행 종로구 교남동 커피원
https://pcmap.place.naver.com/restaurant/267047603/review/visitor#
이번에 찾을 키워드 : 361 / 1179 행 종로구 교남동 마샹

...

https://pcmap.place.naver.com/restaurant/1313743336/review/visitor#
이번에 찾을 키워드 : 803 / 1179 행 종로구 창신2동 스타벅스 동숭길입구점
none
이번에 찾을 키워드 : 804 / 1179 행 종로구 창신2동 총각네붕어빵
https://pcmap.place.naver.com/restaurant/1655421516/review/visitor#
이번에 찾을 키워드 : 805 / 1179 행 종로구 창신2동 바잇더스윗
https://pcmap.place.naver.com/restaurant/1804006479/review/visitor#
이번에 찾을 키워드 : 806 / 1179 행 종로구 창신2동 위치스커피 파스텔드나따 대학로점
https://pcmap.place.naver.com/restaurant/19763664/review/visitor#
이번에 찾을 키워드 : 807 / 1179 행 종로구 창신2동 카페 더블린
https://pcmap.place.naver.com/restaurant/1234568248/review/visitor#
이번에 찾을 키워드 : 808 / 1179 행 종로구 창신2동 멜로우레시피
https://pcmap.place.naver.com/restaurant/1958465960/review/visitor#
이번에 찾을 키워드 : 809 / 1179 행 종로구 창신2동 롤링핀 동대문DDP점
none
이번에 찾을 키워드 : 810 / 1179 행 종로구 창신2동 스탠다드커피
https://pcmap.place.naver.com/restaurant/1479048589/review/visitor#
이번에 찾을 키워드 : 811 / 1179 행 종로구 창신2동 스타벅스 주얼리시티점
none
이번에 찾을 키워드 : 812 / 1179 행 종로구 창신2동 캐치카페 혜화
https://pcmap.place.naver.com/restaurant/1660164157/review/visito

none
이번에 찾을 키워드 : 887 / 1179 행 종로구 숭인1동 24시만화카페
https://pcmap.place.naver.com/restaurant/12975074/review/visitor#
이번에 찾을 키워드 : 888 / 1179 행 종로구 숭인1동 배스킨라빈스 동묘역
none
이번에 찾을 키워드 : 889 / 1179 행 종로구 숭인1동 크레이저커피그룹
https://pcmap.place.naver.com/restaurant/34087284/review/visitor#
이번에 찾을 키워드 : 890 / 1179 행 종로구 숭인1동 제이히든하우스
https://pcmap.place.naver.com/restaurant/1023401640/review/visitor#
이번에 찾을 키워드 : 891 / 1179 행 종로구 숭인1동 테르트르
https://pcmap.place.naver.com/restaurant/1248044853/review/visitor#
이번에 찾을 키워드 : 892 / 1179 행 종로구 숭인1동 나폴레옹과자점 본점
none
이번에 찾을 키워드 : 893 / 1179 행 종로구 숭인1동 심세정
https://pcmap.place.naver.com/restaurant/1972800327/review/visitor#
이번에 찾을 키워드 : 894 / 1179 행 종로구 숭인1동 러시아케익
none
이번에 찾을 키워드 : 895 / 1179 행 종로구 숭인1동 정그리다
https://pcmap.place.naver.com/restaurant/1061286648/review/visitor#
이번에 찾을 키워드 : 896 / 1179 행 종로구 숭인1동 차차티클럽 창신점
https://pcmap.place.naver.com/restaurant/37542108/review/visitor#
이번에 찾을 키워드 : 897 / 1179 행 종로구 숭인1동 더홈서울
https://pcmap.place.naver.com/restaurant/53

none
이번에 찾을 키워드 : 976 / 1179 행 종로구 숭인2동 블루제이커피스페이스
https://pcmap.place.naver.com/restaurant/1351538521/review/visitor#
이번에 찾을 키워드 : 977 / 1179 행 종로구 숭인2동 에스프레소바 오덕새
https://pcmap.place.naver.com/restaurant/1790521600/review/visitor#
이번에 찾을 키워드 : 978 / 1179 행 종로구 숭인2동 쉬즈베이글 고려대점
none
이번에 찾을 키워드 : 979 / 1179 행 종로구 숭인2동 빙달 동대문점
none
이번에 찾을 키워드 : 980 / 1179 행 종로구 숭인2동 Gnoy
https://pcmap.place.naver.com/restaurant/1738165011/review/visitor#
이번에 찾을 키워드 : 981 / 1179 행 종로구 숭인2동 시밀레
https://pcmap.place.naver.com/restaurant/589115279/review/visitor#
이번에 찾을 키워드 : 982 / 1179 행 종로구 숭인2동 스타벅스 황학캐슬점
none
이번에 찾을 키워드 : 983 / 1179 행 종로구 숭인2동 달드
https://pcmap.place.naver.com/restaurant/1484807451/review/visitor#
이번에 찾을 키워드 : 984 / 1179 행 종로구 숭인2동 스윗리버
https://pcmap.place.naver.com/restaurant/1283844701/review/visitor#
이번에 찾을 키워드 : 985 / 1179 행 종로구 숭인2동 공차 왕십리센트라스점
none
이번에 찾을 키워드 : 986 / 1179 행 종로구 숭인2동 잠바주스 동대문두타점
none
이번에 찾을 키워드 : 987 / 1179 행 종로구 숭인2동 보다미
https://pcmap.place.naver.com/restaurant/15350

https://pcmap.place.naver.com/restaurant/716435253/review/visitor#
이번에 찾을 키워드 : 1069 / 1179 행 중구 회현동 반지대학 명동캠퍼스
https://pcmap.place.naver.com/restaurant/36399217/review/visitor#
이번에 찾을 키워드 : 1070 / 1179 행 중구 회현동 스타벅스 신세계본점6F점
none
이번에 찾을 키워드 : 1071 / 1179 행 중구 회현동 컴포즈커피 서울역메트로타워점
none
이번에 찾을 키워드 : 1072 / 1179 행 중구 회현동 스타벅스 신세계본점5F점
none
이번에 찾을 키워드 : 1073 / 1179 행 중구 회현동 스타벅스 서울중앙우체국점
none
이번에 찾을 키워드 : 1074 / 1179 행 중구 회현동 카페오브
none
이번에 찾을 키워드 : 1075 / 1179 행 중구 회현동 토츠오츠
https://pcmap.place.naver.com/restaurant/1282167897/review/visitor#
이번에 찾을 키워드 : 1076 / 1179 행 중구 회현동 caffe b
https://pcmap.place.naver.com/restaurant/253461782/review/visitor#
이번에 찾을 키워드 : 1077 / 1179 행 중구 회현동 로프트
none
이번에 찾을 키워드 : 1078 / 1179 행 중구 회현동 수목티
https://pcmap.place.naver.com/restaurant/1488518175/review/visitor#
이번에 찾을 키워드 : 1079 / 1179 행 중구 회현동 그린밀크커피
https://pcmap.place.naver.com/restaurant/1200487032/review/visitor#
이번에 찾을 키워드 : 1080 / 1179 행 중구 회현동 이디야커피 퇴계로2길점
none
이번에 찾을 키워드 : 1081 / 1179 행 중구 회현동 신라명과

https://pcmap.place.naver.com/restaurant/1794817301/review/visitor#
이번에 찾을 키워드 : 1174 / 1179 행 중구 명동 홀리핸즈커피
https://pcmap.place.naver.com/restaurant/1465770197/review/visitor#
이번에 찾을 키워드 : 1175 / 1179 행 중구 명동 오코아 무교점
https://pcmap.place.naver.com/restaurant/13560571/review/visitor#
이번에 찾을 키워드 : 1176 / 1179 행 중구 명동 아운커피 다동점
https://pcmap.place.naver.com/restaurant/1540359274/review/visitor#
이번에 찾을 키워드 : 1177 / 1179 행 중구 명동 카페화이트랩
none
이번에 찾을 키워드 : 1178 / 1179 행 중구 명동 스타일난다 핑크풀카페
https://pcmap.place.naver.com/restaurant/38692335/review/visitor#
------------------------------ 종료 ------------------------------


In [ ]:
cafe_df
print(len(cafe_df))
print(len(cafe_df.drop_duplicates()))

1179
1179


In [ ]:
cafe_df.to_csv('cafe_link.tsv', sep='\t', encoding = 'utf-8-sig')

In [ ]:
#=================================================================================================================#
#=================================================================================================================#
#=================================================================================================================#
#=================================================================================================================#
#=================================================================================================================#
#=================================================================================================================#

# 리뷰 크롤링

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time, re, os
from bs4 import BeautifulSoup
from tqdm import tqdm 

In [ ]:
driver_path = "C:/chromedriver.exe"
csv_path = "C:/users/user/"

os.chdir(csv_path)

In [ ]:
df = pd.read_csv('cafe_link.csv') 
# 빈칸 제외
df['url'].replace('', np.nan, inplace=True)
df = df.dropna()
df.drop('Unnamed: 0', axis = 1, inplace = True)
print(len(df))
df = df.reset_index()
df

818


,index,place,title,search,url
0,0,종로구 청운효자동,더숲 초소책방,종로구 청운효자동 더숲 초소책방,https://pcmap.place.naver.com/restaurant/16020...
1,1,종로구 청운효자동,에디션덴마크쇼룸,종로구 청운효자동 에디션덴마크쇼룸,https://pcmap.place.naver.com/restaurant/17872...
2,2,종로구 청운효자동,아키비스트,종로구 청운효자동 아키비스트,https://pcmap.place.naver.com/restaurant/17831...
3,3,종로구 청운효자동,서촌스코프,종로구 청운효자동 서촌스코프,https://pcmap.place.naver.com/restaurant/46256...
4,4,종로구 청운효자동,효자베이커리,종로구 청운효자동 효자베이커리,https://pcmap.place.naver.com/restaurant/17898...
...,...,...,...,...,...
813,1172,중구 명동,스마일명품꽈배기 을지로점,중구 명동 스마일명품꽈배기 을지로점,https://pcmap.place.naver.com/restaurant/17948...
814,1173,중구 명동,호두붐 을지로점,중구 명동 호두붐 을지로점,https://pcmap.place.naver.com/restaurant/14657...
815,1174,중구 명동,홀리핸즈커피,중구 명동 홀리핸즈커피,https://pcmap.place.naver.com/restaurant/13560...
816,1175,중구 명동,오코아 무교점,중구 명동 오코아 무교점,https://pcmap.place.naver.com/restaurant/15403...


## 데이터프레임 만든 후 저장

In [ ]:
# 웹드라이버 접속 
driver = webdriver.Chrome(driver_path)

In [ ]:
# 크롤링
cafe_li = []
review_li = []
tag_li = []

print("총 카페수", len(df['title']))
print("-"*10,"Tag 크롤링 시작", "-"*10)

# for j in range(2):     # ========================================================================================== 테스트용
for j in range(len(df)):
    driver.get(df['url'][j]) 
    time.sleep(1)
    for i in range(10):
        # 버튼 누르기
        while True:
            try:
                more_tags = driver.find_element_by_css_selector('a.utrsf._30ord')
                more_tags.send_keys(Keys.ENTER)
            except:
                break
        time.sleep(1)
        titles = driver.find_elements_by_css_selector('div#_title>span._3XamX')
        reviews = driver.find_elements_by_css_selector('li:nth-child({0}) > div.faZHB > a > span.WoYOw'.format(i))
        review_tags = driver.find_elements_by_css_selector('li:nth-child({0}) > div._1aSvN > span.utrsf'.format(i))
        print(titles[0].text)
        cafe_li.append(titles[0].text)
    for review in reviews:
            review_li.append(review.text)
            tags = []
            for tag in review_tags:
                tags.append(tag.text)
            tag_li.append('&'.join(tags))
    print(j+1,"/",len(df['title']),"\t","********** [{}] 리뷰 tag added **********".format(df['title'][j]))
print("-"*10,"Tag 크롤링 종료", "-"*10)

총 카페수 818
---------- Tag 크롤링 시작 ----------
더숲 초소책방
더숲 초소책방
더숲 초소책방
더숲 초소책방
더숲 초소책방
더숲 초소책방
더숲 초소책방
더숲 초소책방
더숲 초소책방
더숲 초소책방
1 / 818 	 ********** [더숲 초소책방] 리뷰 tag added **********
에디션덴마크쇼룸
에디션덴마크쇼룸
에디션덴마크쇼룸
에디션덴마크쇼룸
에디션덴마크쇼룸
에디션덴마크쇼룸
에디션덴마크쇼룸
에디션덴마크쇼룸
에디션덴마크쇼룸
에디션덴마크쇼룸
2 / 818 	 ********** [에디션덴마크쇼룸] 리뷰 tag added **********
아키비스트
아키비스트
아키비스트
아키비스트
아키비스트
아키비스트
아키비스트
아키비스트
아키비스트
아키비스트
3 / 818 	 ********** [아키비스트] 리뷰 tag added **********
서촌스코프
서촌스코프
서촌스코프
서촌스코프
서촌스코프
서촌스코프
서촌스코프
서촌스코프
서촌스코프
서촌스코프
4 / 818 	 ********** [서촌스코프] 리뷰 tag added **********
효자베이커리
효자베이커리
효자베이커리
효자베이커리
효자베이커리
효자베이커리
효자베이커리
효자베이커리
효자베이커리
효자베이커리
5 / 818 	 ********** [효자베이커리] 리뷰 tag added **********
카페이마
카페이마
카페이마
카페이마
카페이마
카페이마
카페이마
카페이마
카페이마
카페이마
6 / 818 	 ********** [카페이마] 리뷰 tag added **********
인왕산 대충유원지
인왕산 대충유원지
인왕산 대충유원지
인왕산 대충유원지
인왕산 대충유원지
인왕산 대충유원지
인왕산 대충유원지
인왕산 대충유원지
인왕산 대충유원지
인왕산 대충유원지
7 / 818 	 ********** [인왕산 대충유원지] 리뷰 tag added **********
스쿠퍼젤라또
스쿠퍼젤라또
스쿠퍼젤라또
스쿠퍼젤라또
스쿠퍼젤라또
스쿠퍼젤라또
스쿠퍼젤라또
스쿠퍼젤라또
스쿠퍼젤라또
스쿠퍼젤

커피투어 경희궁점
커피투어 경희궁점
커피투어 경희궁점
커피투어 경희궁점
커피투어 경희궁점
커피투어 경희궁점
커피투어 경희궁점
커피투어 경희궁점
커피투어 경희궁점
커피투어 경희궁점
64 / 818 	 ********** [커피투어 경희궁점] 리뷰 tag added **********
카페베로나
카페베로나
카페베로나
카페베로나
카페베로나
카페베로나
카페베로나
카페베로나
카페베로나
카페베로나
65 / 818 	 ********** [카페베로나] 리뷰 tag added **********
그린냅
그린냅
그린냅
그린냅
그린냅
그린냅
그린냅
그린냅
그린냅
그린냅
66 / 818 	 ********** [그린냅] 리뷰 tag added **********
그레이커피
그레이커피
그레이커피
그레이커피
그레이커피
그레이커피
그레이커피
그레이커피
그레이커피
그레이커피
67 / 818 	 ********** [그레이커피] 리뷰 tag added **********
홀드미커피
홀드미커피
홀드미커피
홀드미커피
홀드미커피
홀드미커피
홀드미커피
홀드미커피
홀드미커피
홀드미커피
68 / 818 	 ********** [홀드미커피] 리뷰 tag added **********
내자땅콩
내자땅콩
내자땅콩
내자땅콩
내자땅콩
내자땅콩
내자땅콩
내자땅콩
내자땅콩
내자땅콩
69 / 818 	 ********** [내자땅콩] 리뷰 tag added **********
LP소리골
LP소리골
LP소리골
LP소리골
LP소리골
LP소리골
LP소리골
LP소리골
LP소리골
LP소리골
70 / 818 	 ********** [LP소리골] 리뷰 tag added **********
커피붕붕 커피볶는집
커피붕붕 커피볶는집
커피붕붕 커피볶는집
커피붕붕 커피볶는집
커피붕붕 커피볶는집
커피붕붕 커피볶는집
커피붕붕 커피볶는집
커피붕붕 커피볶는집
커피붕붕 커피볶는집
커피붕붕 커피볶는집
71 / 818 	 ********** [커피붕붕 커피볶는집] 리뷰 tag added **********
히든커피
히든커피
히든커피
히든커피
히든

KeyboardInterrupt: 

In [ ]:
review_tag_df = pd.DataFrame((zip(cafe_li, review_li, tag_li)), columns = ['cafe', 'review', 'tag'])
review_tag_df

,cafe,review,tag
0,더숲 초소책방,사장님이 정말 친절하세요 ㅠ 감동 수준;; 바람이 심하게 날리는 편입니다 파라솔이 ...,친절해요&사진이 잘 나와요&집중하기 좋아요&뷰가 좋아요&디저트가 맛있어요
1,더숲 초소책방,예전 초소를 멋지게 바꾸어 카페를 만든 곳이예요 ~워낙 유명한 곳이라 평일에도 차들...,커피가 맛있어요&디저트가 맛있어요&뷰가 좋아요&대화하기 좋아요&친절해요
2,더숲 초소책방,방송을 타서 그런지 정말 사람이 많았아요 왠만하 면 오후 늦게 가시길 추천. 빵 맛...,디저트가 맛있어요&뷰가 좋아요
3,더숲 초소책방,뷰가 좋아 조용히 오전중으로 책 읽고 오려다가 사람이 너무 많아.. 구석진 바위에 ...,커피가 맛있어요&디저트가 맛있어요&뷰가 좋아요&읽을만한 책이 많아요
4,더숲 초소책방,집 근처라서 자주 가는 곳 이긴한데~갈때마다 이 쁘네요~주말엔 사람도 많고 북적이긴...,사진이 잘 나와요&뷰가 좋아요&대화하기 좋아요&좌석이 편해요
...,...,...,...
373,애즈라이크,애즈라이크는 친언니 추천으로 갔는데 너무 맛있었어요! 특히 한우오픈샌드위치와 고구마...,매장이 청결해요&혼밥하기 좋아요&재료가 신선해요&음식이 맛있어요&특별한 메뉴가 있어요
374,애즈라이크,베이컨 / 생연어 오픈 샌드위치 + 오렌지유자에이 드 햇볕드는 창가에서 브런치 조합...,혼밥하기 좋아요&매장이 넓어요&재료가 신선해요&음식이 맛있어요&친절해요
375,애즈라이크,먹어보고 싶은게 많았지만 두명이라 아쉬웠어요ㅠ ㅠㅠ 딱 제일 유명한거 같은 메뉴 두...,친절해요&인테리어가 멋져요&음식이 맛있어요&재료가 신선해요
376,애즈라이크,샌드위치의 고정관념을 박살내버렸습니다 빵을 작 게 칼로 잘라서 야채와 고기와 다 같...,재료가 신선해요&음식이 맛있어요&특별한 메뉴가 있어요&친절해요&인테리어가 멋져요


In [ ]:
review_tag_df.to_csv(csv_path+"review_tag_df.csv",
#            sep='\t', 
           encoding='utf-8-sig')